In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

import analysis as a

p = make_paths(a)

In [ ]:
w = pd.read_table(p['w_file_path'], index_col=0)

h = pd.read_table(p['h_file_path'], index_col=0)

In [ ]:
nmf_map = ccal.NMFMap(
    wt=w.T,
    h=h,
    mds_random_seed=a.NMF_MAP_MDS_RANDOM_SEED,
    wt_pull_power=a.NMF_MAP_WT_PULL_POWER,
    h_pull_power=a.NMF_MAP_H_PULL_POWER,
    plot=False)

for wt_or_h, element_marker_size in (('wt', a.NMF_MAP_WT_ELEMENT_MARKER_SIZE),
                                     ('h', a.NMF_MAP_H_ELEMENT_MARKER_SIZE)):

    nmf_map.plot_nmf_map(
        wt_or_h, title=a.NAME, element_marker_size=element_marker_size)

In [ ]:
for wt_or_h, df, bandwidth_factor, element_marker_size in (
    ('wt',
     pd.DataFrame(
         nmf_map.wt, index=nmf_map.nodes, columns=nmf_map.wt_elements),
     a.NMF_MAP_WT_BANDWIDTH_FACTOR, a.NMF_MAP_WT_ELEMENT_MARKER_SIZE),
    ('h',
     pd.DataFrame(nmf_map.h, index=nmf_map.nodes, columns=nmf_map.h_elements),
     a.NMF_MAP_H_BANDWIDTH_FACTOR, a.NMF_MAP_H_ELEMENT_MARKER_SIZE),
):

    hcc__k_x_column_file_path = p['{}_hcc__k_x_column_file_path'.format(
        wt_or_h)]

    if os.path.isfile(hcc__k_x_column_file_path):

        hcc__k_x_column = pd.read_table(hcc__k_x_column_file_path, index_col=0)

    else:

        if df.shape[1] < 1e3:

            distance_function = ccal.compute_information_distance

        else:

            distance_function = compute_correlation_distance

        distance__column_x_column, k_hcc_distance_linkage, k_hcc_distance_linkage_ccc, hcc__k_x_column = ccal.hierarchical_consensus_cluster_with_multiple_k(
            df,
            a.HCC_KS,
            n_job=len(a.HCC_KS),
            distance_function=distance_function,
            n_clustering=100,
            plot_df=False,
            directory_path=p['{}_hcc_directory_path'.format(wt_or_h)])

    for hcc_k in a.HCC_KS[::3]:

        element_state = hcc__k_x_column.loc['K{}'.format(hcc_k)]

        if element_state.value_counts().min() < 3:

            print(
                'Skipping {} because it has at least 1 state with less than 3 elements ...'.
                format(hcc_k))

        else:

            nmf_map.set_element_states(
                wt_or_h, element_state, bandwidth_factor=bandwidth_factor)

            nmf_map.plot_nmf_map(
                wt_or_h,
                title=a.NAME,
                element_marker_size=element_marker_size,
                html_file_path='{}/map_{}.html'.format(
                    p['{}_map_directory_path'.format(wt_or_h)], hcc_k))